Em vez de estar sempre a usar os thresholds de 90/180, usar um slider. Por exemplo quando o slider esta nos 45, ele vai fazer o processamento todo (o pipeline todo) para me dar a imagem que em vez de ser com thresholds de 90/180, seja com 45/90. 

Tenho que começar a agrupar isto por funções, de maneira a que qnd altero o 1º parametro, tem-se um input e esse input vai fazer com que a imagem do canny vá mudar e vamos correr o algoritmo para as imagens em questão

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import netCDF4 as nc
import cv2
import datetime
import xarray as xr
import os
plt.rcParams["figure.figsize"] = 10, 8
%matplotlib inline

In [3]:
current_path = os.getcwd()
current_path

'/home/luisfigueiredo/JUNO/notebooks'

In [4]:
from pathlib import Path

data_folder = os.path.join(current_path,"../data")
data_folder

'/home/luisfigueiredo/JUNO/notebooks/../data'

In [5]:
nc_path = os.path.join(data_folder, "IBI2014-2019.nc")
ds = nc.Dataset(nc_path)
data = xr.load_dataset(nc_path)

In [6]:
print(ds)     # informações relativas ao ficheiro netCDF

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    source: CMEMS IBI-MFC
    institution: Puertos del Estado (PdE) - Mercator-Ocean (MO)
    references: http://marine.copernicus.eu
    title: CMEMS IBI REANALYSIS: DAILY PHYSICAL PRODUCTS 
    easting: longitude
    northing: latitude
    domain_name: IBI12
    FROM_ORIGINAL_FILE__field_type: mean
    field_date: 20191224
    FROM_ORIGINAL_FILE__longitude_min: -19.f
    FROM_ORIGINAL_FILE__longitude_max: 5.f
    FROM_ORIGINAL_FILE__latitude_min: 26.f
    FROM_ORIGINAL_FILE__latitude_max: 56.f
    z_min: 0.50576f
    z_max: 5698.061f
    contact: mailto: servicedesk.cmems@mercator-ocean.eu
    bulletin_date: 2020-12-01
    bulletin_type: Reanalysis
    _CoordSysBuilder: ucar.nc2.dataset.conv.CF1Convention
    comment: 
    history: Data extracted from dataset http://localhost:8080/thredds/dodsC/cmems_mod_ibi_phy_my_0.083deg-3D_P1D-m
    dimensions(sizes): depth(1), latitude(1

In [7]:
#Converter o ficheiro netCDF para uma dataframe
datadf = data.to_dataframe()
df2014_2019 = datadf.reset_index()

In [8]:
df2014_2019 = df2014_2019.drop(['depth'], axis=1)     #dropar coluna 'depth' (não é necessária pq é sempre igual)
df2014_2019   #os dados .nc estão agora no formato de uma dataframe

,latitude,longitude,time,thetao
0,35.0,-19.0,2014-01-01 12:00:00,18.316000
1,35.0,-19.0,2014-01-02 12:00:00,18.309000
2,35.0,-19.0,2014-01-03 12:00:00,18.316000
3,35.0,-19.0,2014-01-04 12:00:00,18.273001
4,35.0,-19.0,2014-01-05 12:00:00,18.230000
...,...,...,...,...
44660611,45.0,-5.0,2019-12-20 12:00:00,13.619000
44660612,45.0,-5.0,2019-12-21 12:00:00,13.559000
44660613,45.0,-5.0,2019-12-22 12:00:00,13.458000
44660614,45.0,-5.0,2019-12-23 12:00:00,13.400000


In [9]:
df2014_2019 = df2014_2019[['time', 'latitude', 'longitude', 'thetao']]    #reorganizar as colunas da dataframe
df2014_2019

,time,latitude,longitude,thetao
0,2014-01-01 12:00:00,35.0,-19.0,18.316000
1,2014-01-02 12:00:00,35.0,-19.0,18.309000
2,2014-01-03 12:00:00,35.0,-19.0,18.316000
3,2014-01-04 12:00:00,35.0,-19.0,18.273001
4,2014-01-05 12:00:00,35.0,-19.0,18.230000
...,...,...,...,...
44660611,2019-12-20 12:00:00,45.0,-5.0,13.619000
44660612,2019-12-21 12:00:00,45.0,-5.0,13.559000
44660613,2019-12-22 12:00:00,45.0,-5.0,13.458000
44660614,2019-12-23 12:00:00,45.0,-5.0,13.400000
